In [1]:
import cv2, io, time, imutils, datetime
import traceback
from imutils.video import VideoStream
import numpy as np
from classes import imagenet_classes
from ovmsclient import make_grpc_client
import opencv_jupyter_ui as jcv2

from utils import crop_resize, draw_label, prep_image
from IPython.display import display, Image
import ipywidgets as widgets
import threading

In [2]:
GRPC_URL = "localhost:9000"
RTSP_URL = "rtsp://localhost:8554/server"
WIDTH = 640
HEIGHT = 480
FPS = 30

In [3]:
grpc_client = make_grpc_client(GRPC_URL)
print("* gRPC Socket succesfully opened")

* gRPC Socket succesfully opened


In [4]:
rtsp_stream = VideoStream(RTSP_URL).start()
print("* RTSP Stream succesfully opened")

* RTSP Stream succesfully opened


In [5]:
def resnet(client, img):
    output = client.predict({"0": img}, "resnet")
    result_index = np.argmax(output[0])
    predicted_class = imagenet_classes[result_index]
    return predicted_class

In [ ]:
while True:
    try: 
        frame = rtsp_stream.read()
        try: 
            if frame is not None:
                aux_frame = prep_image(frame, WIDTH, HEIGHT)
                result = resnet(grpc_client, aux_frame)
                draw_label(frame, str(result), (20,20), (255,0,0))
                title = "Prediction OUTPUT"
                jcv2.imshow(title, frame)
            else: 
                continue 
                
            time.sleep(1/32)
        except Exception as e:
            print("error while running inference", e)
            print(traceback.format_exc())
          
    except Exception as e:
        print("error while grabbing RTSP stream", e)